In [ ]:
import functools

# Definitions for both keypads. Made into tuples to cache them later
numeric_keypad = [['7', '8', '9'], ['4', '5', '6'], ['1', '2', '3'], [None, '0', 'A']]
numeric_keypad = tuple(tuple(row) for row in numeric_keypad)
directional_keypad = [[None, '^', 'A'], ['<', 'v', '>']]
directional_keypad = tuple(tuple(row) for row in directional_keypad)

# All directions, in y,x coordiantes. 
directions = {'^': (-1, 0), 'v': (1, 0), '<': (0, -1), '>': (0, 1)} # Diff in y, x

# Decided to pre-invert the keypads for easier access later... 
inverted_numeric = {
            numeric_keypad[y][x]: (y, x) for x in range(len(numeric_keypad[0])) for y in range(len(numeric_keypad))
        }
inverted_directional = {
            directional_keypad[y][x]: (y, x) for x in range(len(directional_keypad[0])) for y in range(len(directional_keypad))
        }

@functools.cache
def search(keypad, start_pos, end_pos):
    """"
        Simply returns all possible shortest paths from start_pos to end_pos.

        They are returned as strings, where each character is a direction as given on the passed keyboard. 
    """
    possible_paths = list()
    if start_pos == end_pos:
        return [""]
    to_expand = [(start_pos, "")] # (position, string)
    
    while to_expand:
        next_expand = list()
        for pos, so_far in to_expand: 
            for direction in "><v^": 
                dy, dx = directions[direction]
                nextpos = (pos[0] + dy, pos[1] + dx)
                if 0 <= nextpos[0] < len(keypad) and 0 <= nextpos[1] < len(keypad[0]) and keypad[nextpos[0]][nextpos[1]] != None: 
                    if nextpos == end_pos: 
                        possible_paths.append(so_far + direction)
                    else: 
                        next_expand.append((nextpos, so_far + direction))
        if len(possible_paths) > 0: 
            return possible_paths
        else: 
            to_expand = next_expand


def get_path_for_key_combination(wanted, keypad, start_position): 
    """ Returns all possible paths for a given key combination 

    You can pass both keypads, and can pass what output combination you want to achieve. 

    You can also pass the start position for where the robot finger is hovering... 
    """
    inverted_keys = {
        keypad[y][x]: (y, x) for x in range(len(keypad[0])) for y in range(len(keypad))
    }

    possible_paths = [""]
    for next_key in wanted: 
        end_position = inverted_keys[next_key]
        
        possible_parts = search(keypad, start_position, end_position)
        new_possible = list()
        # Note: this exponentially grows and is the reason part 2 doesn't work with this code... 
        for existing in possible_paths: 
            for part in possible_parts: 
                new_possible.append(existing + part + 'A')
        possible_paths = new_possible
        start_position = end_position
    return possible_paths

def execute_code(code, keypad, start_pos): 
    """
    Helper function to execute a code on a keypad. 
    Needed as I first didn't understand why my answer to part1 was not correct... 
    """
    typed_code = ""
    for c in code: 
        if c == 'A': 
            typed_code += keypad[start_pos[0]][start_pos[1]]
        else: 
            dy, dx = directions[c]
            start_pos = (start_pos[0] + dy, start_pos[1] + dx)
            if keypad[start_pos[0]][start_pos[1]] == None: 
                raise ValueError("robot exploded")
        
    return typed_code

def part_1(code): 
    """
    For part 1 I actually generate the paths the robot would need to press... 

    However, that approach breaks down for part 2, as the number of paths grows exponentially.
    """
    options = get_path_for_key_combination(code, keypad = numeric_keypad, start_position = (3, 2))
    
    for _ in range(2): 
        options.sort(key=lambda x: len(x))
        next_options = list()
        for p1 in options: 
            p2 = get_path_for_key_combination(p1, keypad = directional_keypad, start_position = (0,2))
            next_options.extend(p2)
        options = next_options
    return options


@functools.cache
def get_length_path(start, end, depth, keypad): 
    """"
        For part 2 I needed to determine the length of the path instead of getting the actual path itself... 

        That can be a recursive function which caches the length of subpaths to avoid recalculating them.
    """
    if start == end: 
        return 1 # If already there we just need to press the A button... 
    
    possible_paths = search(keypad, start, end)
    
    # Determine the length of each sequence one level down... 
    lengths = list()
    for p in possible_paths:     
        p += 'A'
        if depth == 1: 
            lengths.append(len(p))
        else:
            next_start = inverted_directional['A']
            length_this_path = 0
            for character in p: 
                next_end = inverted_directional[character]
                length_this_path += get_length_path(next_start, next_end, depth - 1, directional_keypad)
                next_start = next_end
            lengths.append(length_this_path)
    
    # Return the shortest path of all possible paths...
    return min(lengths)
        
codes = [code.strip() for code in open('inputs/day21.txt').readlines()]
sum_complexities_p1 = 0
sum_complexities_p2 = 0
for code in codes:
    numerical_part = int(code[:3])

    # For part 1 we just 
    p = part_1(code.strip())
    p.sort(key=lambda x: len(x))
    sum_complexities_p1 += numerical_part * len(p[0])

    total_length = 0
    startpos = inverted_numeric['A']
    for char in code: 
        endpos = inverted_numeric[char]
        total_length += get_length_path(startpos, endpos, 25+1, numeric_keypad)
        startpos = endpos

    sum_complexities_p2 += total_length * numerical_part
print('Part 1', sum_complexities_p1)
print('Part 2', sum_complexities_p2)


Part 1 123096
Part 2 154517692795352


: 